In [23]:
import pandas as pd
import numpy as np
import kagglehub
import os
import datetime
import exchange_calendars as ecals
from zoneinfo import ZoneInfo

In [24]:
path = kagglehub.dataset_download("leukipp/hourly-stock-prices")
csv = os.path.join(path, '2021', 'AMD' + '.csv')
df = pd.read_csv(csv)
df

,ticker,name,date,open,high,low,close,adjusted close,volume
0,AMD,"Advanced Micro Devices, Inc. - Common Stock",2021-01-04 04:00:00-05:00,92.120000,92.250000,91.830000,91.830000,91.830000,0
1,AMD,"Advanced Micro Devices, Inc. - Common Stock",2021-01-04 05:00:00-05:00,91.700000,92.340000,91.700000,92.290000,92.290000,0
2,AMD,"Advanced Micro Devices, Inc. - Common Stock",2021-01-04 06:00:00-05:00,92.030000,92.600000,92.010000,92.410000,92.410000,0
3,AMD,"Advanced Micro Devices, Inc. - Common Stock",2021-01-04 07:00:00-05:00,92.400000,92.410000,92.000000,92.150000,92.150000,0
4,AMD,"Advanced Micro Devices, Inc. - Common Stock",2021-01-04 08:00:00-05:00,92.200000,92.260000,92.030000,92.130000,92.130000,0
...,...,...,...,...,...,...,...,...,...
4069,AMD,"Advanced Micro Devices, Inc. - Common Stock",2021-12-30 13:30:00-05:00,146.988007,147.429993,146.179993,146.639893,146.639893,4019186
4070,AMD,"Advanced Micro Devices, Inc. - Common Stock",2021-12-30 14:30:00-05:00,146.630005,146.770004,145.789993,146.070007,146.070007,4560718
4071,AMD,"Advanced Micro Devices, Inc. - Common Stock",2021-12-30 15:30:00-05:00,146.065002,146.240005,144.850006,145.089996,145.089996,4974318
4072,AMD,"Advanced Micro Devices, Inc. - Common Stock",2021-12-30 16:00:00-05:00,145.150000,147.100000,137.170900,146.230000,146.230000,0


In [25]:
df['date'] = pd.to_datetime(df['date'])

/tmp/ipykernel_316622/3532345252.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['date'] = pd.to_datetime(df['date'])


In [26]:
target_datetime = pd.Timestamp("2021-03-11 10:30:00-05:00")
df[df["date"] >= target_datetime].iloc[0]
df

,ticker,name,date,open,high,low,close,adjusted close,volume
0,AMD,"Advanced Micro Devices, Inc. - Common Stock",2021-01-04 04:00:00-05:00,92.120000,92.250000,91.830000,91.830000,91.830000,0
1,AMD,"Advanced Micro Devices, Inc. - Common Stock",2021-01-04 05:00:00-05:00,91.700000,92.340000,91.700000,92.290000,92.290000,0
2,AMD,"Advanced Micro Devices, Inc. - Common Stock",2021-01-04 06:00:00-05:00,92.030000,92.600000,92.010000,92.410000,92.410000,0
3,AMD,"Advanced Micro Devices, Inc. - Common Stock",2021-01-04 07:00:00-05:00,92.400000,92.410000,92.000000,92.150000,92.150000,0
4,AMD,"Advanced Micro Devices, Inc. - Common Stock",2021-01-04 08:00:00-05:00,92.200000,92.260000,92.030000,92.130000,92.130000,0
...,...,...,...,...,...,...,...,...,...
4069,AMD,"Advanced Micro Devices, Inc. - Common Stock",2021-12-30 13:30:00-05:00,146.988007,147.429993,146.179993,146.639893,146.639893,4019186
4070,AMD,"Advanced Micro Devices, Inc. - Common Stock",2021-12-30 14:30:00-05:00,146.630005,146.770004,145.789993,146.070007,146.070007,4560718
4071,AMD,"Advanced Micro Devices, Inc. - Common Stock",2021-12-30 15:30:00-05:00,146.065002,146.240005,144.850006,145.089996,145.089996,4974318
4072,AMD,"Advanced Micro Devices, Inc. - Common Stock",2021-12-30 16:00:00-05:00,145.150000,147.100000,137.170900,146.230000,146.230000,0


In [27]:
def is_nasdaq_open(date: datetime) -> bool:
        '''
            method for checking if nasdaq is open in the given datetime
        '''
        calendar = ecals.get_calendar("XNYS")
        if not isinstance(date, pd.Timestamp):
            date = pd.Timestamp(date)
            
        if not calendar.is_session(date.date()):
            return False

        open_time, close_time = calendar.session_open_close(date.date())
        open_time, close_time = open_time.astimezone("US/Eastern"), close_time.astimezone("US/Eastern")
        return open_time.time() <= date.time() <= close_time.time()
    
df = df[df['date'].apply(is_nasdaq_open)].reset_index(drop=True)
df = df.drop(columns=['ticker', 'name'])
df


,date,open,high,low,close,adjusted close,volume
0,2021-01-04 09:30:00-05:00,92.139999,96.059998,91.400002,95.879997,95.879997,17147367
1,2021-01-04 10:30:00-05:00,95.889999,95.948997,92.930000,93.180000,93.180000,9583950
2,2021-01-04 11:30:00-05:00,93.180000,93.535004,90.915001,91.349998,91.349998,7871625
3,2021-01-04 12:30:00-05:00,91.309998,91.989998,90.930000,91.870003,91.870003,4733033
4,2021-01-04 13:30:00-05:00,91.849998,92.220001,91.620003,91.927299,91.927299,3675526
...,...,...,...,...,...,...,...
1951,2021-12-30 12:30:00-05:00,148.174896,148.192703,146.740005,146.979996,146.979996,4104248
1952,2021-12-30 13:30:00-05:00,146.988007,147.429993,146.179993,146.639893,146.639893,4019186
1953,2021-12-30 14:30:00-05:00,146.630005,146.770004,145.789993,146.070007,146.070007,4560718
1954,2021-12-30 15:30:00-05:00,146.065002,146.240005,144.850006,145.089996,145.089996,4974318


In [28]:
df.to_csv('../../data/stock.csv')